In [68]:
import os
import time
#import maritalk
import openai
import pyconll
import json
from sklearn import metrics
from tqdm import tqdm

In [72]:
with open('gptkey2.txt') as f:
    gpt_key = f.readlines()[0]
openai.api_key = gpt_key

In [ ]:
def askGPT2(prompt, temperature=0.7):
    retries = 3
    while retries > 0:
        try:    
            messages = [{"role": "user", "content": prompt}]
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=messages,
                temperature=temperature,
            )
        except Exception as e: 
            if e:
                print(e)
                retries -= 1
                time.sleep(5)
            else:
                raise e
    return response

In [98]:
def askGPT(prompt, temperature=0.7):
    openai.api_key = gpt_key
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        request_timeout=120,
        messages=messages,
        temperature=temperature,
    )
    time.sleep(2)
    return response

In [4]:
resp = askGPT("Quanto é 25 + 27?", 0.9)
print(resp)

{
  "id": "chatcmpl-8GriJE6K30LQQvscNDimIKSPIcfIM",
  "object": "chat.completion",
  "created": 1699030415,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "25 + 27 = 52"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 16,
    "completion_tokens": 7,
    "total_tokens": 23
  }
}


In [10]:
a = resp['choices'][0]['message']['content']
print(a)

25 + 27 = 52


In [11]:
# reading first 1020 dataset sentences
frases = pyconll.load_from_file('data/porttinari-base/Porttinari-base_test.conllu')
reviews = []
for sent in frases[:1020]:
    lista_ud=[]
    for token in sent:
        lista_ud.append([token.form, token.xpos, token.upos, token.deprel])
    reviews.append(lista_ud)

In [36]:
promptStart = []
promptStart.append({
    "role": "system",
    "content":"Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):"
})
for review in reviews[:10]:
    entrada = ""
    saida = ""
    for token in review:
        if not token[2]:
            token[2] = 'None'
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    promptStart.append({
        "role": "user",
        "content": entrada
    })
    promptStart.append({
        "role": "system",
        "content": saida
    })
promptStart

[{'role': 'system',
  'content': 'Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies):'},
 {'role': 'user', 'content': 'O Capitão América também bajulou o tucano .'},
 {'role': 'system',
  'content': 'O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT'},
 {'role': 'user',
  'content': 'A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .'},
 {'role': 'system',
  'content': 'A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT'},
 {'role': 'user',
  'content': 'No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .'},
 {'role': 'system',
  'content': 'No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN c

In [42]:
head = "Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) conforme os exemplos abaixo:\n\n"
promptStart = head
for review in reviews[:10]:
    entrada = "Entrada: "
    saida  = "Saida: "
    for token in review:
        if not token[2]:
            token[2] = 'None'
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    promptStart += entrada + "\n" + saida + "\n\n"
print(promptStart)

Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) conforme os exemplos abaixo:

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .
Saida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT

Entrada: Os sons indesejáveis emitidos por uma porta , por exemplo , são e

In [50]:
punctList = ['"', ',', '.', '[', ']', '?','!']
for temp in range(0,10):
    if temp:
        temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for revID, review in enumerate(tqdm(reviews[20:40])):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "        
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "        
        #prompt.append({
        #    "role": "user",
        #    "content": entrada
        #})
        fileName = f'resultados/gpt/gpt_t{temp}_{revID}.json'
        if os.path.isfile(fileName):
            with open(fileName) as file:
                tagsPred = json.load(file)
        else:            
            tries = 0
            while True:
                if tries > 10:
                    errors += 1
                    totalTries.append(tries)
                    with open(f'resultados/gpt/gpt_test_erro_t{temp}_{revID}.log', 'w') as f:
                        f.write(prompt)
                        f.write('======')
                        f.write(answer+'\n\n')
                        f.write('gold'+ str(goldSent) + str(len(goldSent))+'\n')
                        f.write(str(goldTokens)+'\n')
                        f.write('pred' + str(tagsSent) + str(len(tagsSent))+'\n')
                        f.write(str(tokens)+'\n')
                        f.write('\n\n')
                    tagsPred = ['None'] * len(goldSent)
                    #dsdaasd
                    break
                resp = askGPT(prompt, temperature=temp)
                answer = resp['choices'][0]['message']['content']
                tries += 1
                try:
                    tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                    tokensSent = []
                    tagsSent = []
                    for token in tokens:
                        if '/' in token:
                            tagsSent.append(token.split('/')[1])
                            tokensSent.append(token.split('/')[0])
                        else:
                            tagsSent.append('None')
                            tokensSent.append(token)
                except Exception as error:
                    continue
                idx = 0
                erroIdx = 0
                tagsPred = []
                for token, tag in zip(goldTokens, goldSent):
                    if idx >= len(tokensSent):
                        tagsPred.append('None')
                        erroIdx += 1
                        continue           
                    if tokensSent[idx].lower() == token.lower():
                        tagsPred.append(tagsSent[idx])
                    else:
                        if idx == 0:
                            while (tokensSent[idx].lower() != token.lower() and idx < 5):
                                idx += 1
                                if idx >= len(tokensSent):
                                    idx -= 1
                                    break
                        if tokensSent[idx].lower() == token.lower():
                            tagsPred.append(tagsSent[idx])
                        else:
                            tagsPred.append('None')
                            erroIdx += 1
                            continue
                    idx += 1
                if len(tagsPred) == len(goldSent) and erroIdx < 10:
                    totalTries.append(tries)
                    break
        with open(fileName, 'w') as f:
            json.dump(tagsPred, f)
        pred += tagsPred
        #ksdfhfkdh
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/gpt_test_t{temp}.json', 'w') as f:
        json.dump(pred, f)
    #with open(f'resultados/gold.json', 'w') as f:
    #    json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

temp 0


100%|███████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 8113.56it/s]


gold 315 pred 315 errors 0
              precision    recall  f1-score   support

         ADJ       0.89      0.94      0.92        18
         ADP       0.91      0.96      0.93        45
         ADV       0.90      0.69      0.78        13
         AUX       0.94      0.94      0.94        16
       CCONJ       1.00      1.00      1.00         8
         DET       0.94      0.94      0.94        47
        INTJ       1.00      1.00      1.00         1
        NOUN       1.00      0.95      0.98        62
         NUM       1.00      1.00      1.00         5
        None       0.94      0.71      0.81        21
        PRON       0.70      0.88      0.78        16
       PROPN       1.00      1.00      1.00        22
       SCONJ       0.62      1.00      0.77         5
         SYM       1.00      1.00      1.00         1
        VERB       0.94      0.97      0.96        35

    accuracy                           0.93       315
   macro avg       0.92      0.93      0.92       315

100%|█████████████████████████████████████████████████████████████████| 20/20 [04:36<00:00, 13.84s/it]


gold 315 pred 315 errors 0
              precision    recall  f1-score   support

         ADJ       0.89      0.94      0.92        18
         ADP       0.88      0.96      0.91        45
         ADV       0.90      0.69      0.78        13
         AUX       0.88      0.94      0.91        16
       CCONJ       1.00      1.00      1.00         8
         DET       0.93      0.91      0.92        47
        INTJ       1.00      1.00      1.00         1
        NOUN       1.00      0.95      0.98        62
         NUM       1.00      1.00      1.00         5
        None       0.93      0.62      0.74        21
        PRON       0.67      0.88      0.76        16
       PROPN       1.00      1.00      1.00        22
       SCONJ       0.62      1.00      0.77         5
         SYM       1.00      1.00      1.00         1
        VERB       0.94      0.94      0.94        35

    accuracy                           0.91       315
   macro avg       0.91      0.92      0.91       315

...

100%|█████████████████████████████████████████████████████████████████| 20/20 [04:55<00:00, 14.79s/it]
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter 

gold 315 pred 315 errors 0
              precision    recall  f1-score   support

         ADJ       0.85      0.94      0.89        18
         ADP       0.93      0.93      0.93        45
      ADPles       0.00      0.00      0.00         0
         ADV       0.89      0.62      0.73        13
         AUX       0.83      0.94      0.88        16
       CCONJ       1.00      1.00      1.00         8
         DET       0.95      0.89      0.92        47
        INTJ       1.00      1.00      1.00         1
        NOUN       0.98      0.94      0.96        62
         NUM       1.00      1.00      1.00         5
        None       0.89      0.76      0.82        21
        PRON       0.62      0.81      0.70        16
       PROPN       1.00      1.00      1.00        22
       SCONJ       0.56      1.00      0.71         5
         SYM       1.00      1.00      1.00         1
        VERB       0.94      0.91      0.93        35

    accuracy                           0.90       315

100%|█████████████████████████████████████████████████████████████████| 20/20 [04:58<00:00, 14.95s/it]

gold 315 pred 315 errors 0
              precision    recall  f1-score   support

         ADJ       0.77      0.94      0.85        18
         ADP       0.90      0.96      0.92        45
      ADPles       0.00      0.00      0.00         0
         ADV       1.00      0.69      0.82        13
         AUX       0.87      0.81      0.84        16
       CCONJ       1.00      1.00      1.00         8
         DET       0.93      0.85      0.89        47
        INTJ       1.00      1.00      1.00         1
        NOUN       1.00      0.90      0.95        62
         NUM       0.71      1.00      0.83         5
        None       0.88      0.67      0.76        21
        PRON       0.57      0.81      0.67        16
       PROPN       1.00      1.00      1.00        22
       SCONJ       1.00      1.00      1.00         5
         SYM       1.00      1.00      1.00         1
        VERB       0.87      0.94      0.90        35

    accuracy                           0.89       315


/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 0.0 full experiment

In [106]:


punctList = ['"', ',', '.', '[', ']', '?','!']
for temp in range(0,1):
    if temp:
        temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for revID, review in enumerate(tqdm(reviews[20:])):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "        
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "        
        #prompt.append({
        #    "role": "user",
        #    "content": entrada
        #})
        fileName = f'resultados/gpt/gpt_t{temp}_{revID}.json'
        if os.path.isfile(fileName):
            with open(fileName) as file:
                tagsPred = json.load(file)
        else:            
            tries = 0
            while True:
                if tries > 10:
                    errors += 1
                    totalTries.append(tries)
                    with open(f'resultados/gpt/gpt_erro_t{temp}_{revID}.log', 'w') as f:
                        f.write(prompt)
                        f.write('======')
                        f.write(answer+'\n\n')
                        f.write('gold'+ str(goldSent) + str(len(goldSent))+'\n')
                        f.write(str(goldTokens)+'\n')
                        f.write('pred' + str(tagsSent) + str(len(tagsSent))+'\n')
                        f.write(str(tokens)+'\n')
                        f.write('\n\n')
                    tagsPred = ['None'] * len(goldSent)
                    #dsdaasd
                    break
                resp = askGPT(prompt, temperature=temp)
                answer = resp['choices'][0]['message']['content']
                tries += 1
                try:
                    tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                    tokensSent = []
                    tagsSent = []
                    for token in tokens:
                        if '/' in token:
                            tagsSent.append(token.split('/')[1])
                            tokensSent.append(token.split('/')[0])
                        else:
                            tagsSent.append('None')
                            tokensSent.append(token)
                except Exception as error:
                    continue
                idx = 0
                erroIdx = 0
                tagsPred = []
                for token, tag in zip(goldTokens, goldSent):
                    if idx >= len(tokensSent):
                        tagsPred.append('None')
                        erroIdx += 1
                        continue           
                    if tokensSent[idx].lower() == token.lower():
                        tagsPred.append(tagsSent[idx])
                    else:
                        if idx == 0:
                            while (tokensSent[idx].lower() != token.lower() and idx < 5):
                                idx += 1
                                if idx >= len(tokensSent):
                                    idx -= 1
                                    break
                        if tokensSent[idx].lower() == token.lower():
                            tagsPred.append(tagsSent[idx])
                        else:
                            tagsPred.append('None')
                            erroIdx += 1
                            continue
                    idx += 1
                if len(tagsPred) == len(goldSent) and erroIdx < 10:
                    totalTries.append(tries)
                    break
        with open(fileName, 'w') as f:
            json.dump(tagsPred, f)
        pred += tagsPred
        #ksdfhfkdh
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/gpt_t{temp}.json', 'w') as f:
        json.dump(pred, f)
    #with open(f'resultados/gold.json', 'w') as f:
    #    json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

temp 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [14:56<00:00,  1.12it/s]

gold 18690 pred 18690 errors 0
              precision    recall  f1-score   support

           "       0.00      0.00      0.00         0
           (       0.00      0.00      0.00         0
           )       0.00      0.00      0.00         0
         ADJ       0.82      0.97      0.89       996
         ADP       0.85      0.93      0.89      2943
         ADV       0.91      0.87      0.89       759
         AUX       0.79      0.89      0.84       592
       CCONJ       0.99      0.95      0.97       497
         DET       0.91      0.94      0.93      2880
        INTJ       0.75      1.00      0.86         3
        NOUN       0.98      0.96      0.97      3757
         NUM       0.96      0.87      0.91       364
        None       0.83      0.58      0.68      1208
        PRON       0.81      0.78      0.80       771
       PROPN       0.98      0.93      0.95      1290
       PUNCT       1.00      0.87      0.93       222
       SCONJ       0.65      0.97      0.78       


/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mateus.machado/venvs/propor/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home

In [107]:
print(totalTries)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [109]:
len(pred)

18690

In [110]:
len(gold)

18690

In [111]:
with open(f'resultados/gpt_final.json', 'w') as f:
    json.dump(pred, f)

In [83]:
len(prompt)

3265

In [84]:
prompt

'Atuando como linguista e sem efetuar correções ou alterações no texto, faça a análise morfossintática das frases seguindo a anotação UD (Universal Dependencies) conforme os exemplos abaixo:\n\nEntrada: O Capitão América também bajulou o tucano .\nSaida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT\n\nEntrada: A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .\nSaida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT\n\nEntrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .\nSaida: No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNCT\n\nEntrada: Os sons indesejáveis emitidos por uma porta , por exe

In [46]:
pred

['PRON', 'PRON', 'AUX', 'VERB', 'NOUN', 'CCONJ', 'AUX', 'VERB', 'DET', 'NOUN']

In [37]:
punctList = ['"', ',', '.', '[', ']', '?','!']
for temp in range(0,1):
    if temp:
        temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for revID, review in enumerate(tqdm(reviews[21:22])):
        prompt = promptStart.copy()
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt.append({
            "role": "user",
            "content": entrada
        })
        fileName = f'resultados/gpt/gpt_t{temp}_{revID}.json'
        if os.path.isfile(fileName):
            with open(fileName) as file:
                tagsPred = json.load(file)
        else:            
            tries = 0
            while True:
                if tries > 0:
                    errors += 1
                    totalTries.append(tries)
                    with open(f'resultados/gpt/gpt_test_erro_t{temp}_{revID}.log', 'w') as f:
                        f.write(prompt)
                        f.write('======')
                        f.write(answer+'\n\n')
                        f.write('gold'+ str(goldSent) + str(len(goldSent))+'\n')
                        f.write(str(goldTokens)+'\n')
                        f.write('pred' + str(tagsSent) + str(len(tagsSent))+'\n')
                        f.write(str(tokens)+'\n')
                        f.write('\n\n')
                    tagsPred = ['None'] * len(goldSent)
                    #dsdaasd
                    break
                resp = askGPT(prompt, temperature=temp)
                answer = resp['choices'][0]['message']['content']
                tries += 1
                try:
                    tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                    tokensSent = []
                    tagsSent = []
                    for token in tokens:
                        if '/' in token:
                            tagsSent.append(token.split('/')[1])
                            tokensSent.append(token.split('/')[0])
                        else:
                            tagsSent.append('None')
                            tokensSent.append(token)
                except Exception as error:
                    continue
                idx = 0
                erroIdx = 0
                tagsPred = []
                for token, tag in zip(goldTokens, goldSent):
                    if idx >= len(tokensSent):
                        tagsPred.append('None')
                        erroIdx += 1
                        continue           
                    if tokensSent[idx].lower() == token.lower():
                        tagsPred.append(tagsSent[idx])
                    else:
                        if idx == 0:
                            while (tokensSent[idx].lower() != token.lower() and idx < 5):
                                idx += 1
                                if idx >= len(tokensSent):
                                    idx -= 1
                                    break
                        if tokensSent[idx].lower() == token.lower():
                            tagsPred.append(tagsSent[idx])
                        else:
                            tagsPred.append('None')
                            erroIdx += 1
                            continue
                    idx += 1
                if len(tagsPred) == len(goldSent) and erroIdx < 10:
                    totalTries.append(tries)
                    break
        with open(fileName, 'w') as f:
            json.dump(tagsPred, f)
        pred += tagsPred
        #ksdfhfkdh
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/gpt_test_t{temp}.json', 'w') as f:
        json.dump(pred, f)
    #with open(f'resultados/gold.json', 'w') as f:
    #    json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

temp 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1555.17it/s]

gold 10 pred 7 errors 0


ValueError: Found input variables with inconsistent numbers of samples: [10, 7]

In [38]:
pred

['None', 'None', 'None', 'None', 'None', 'None', 'None']

In [39]:
answer

'Desculpe, mas não entendi o contexto ou a intenção por trás da repetição do texto. Posso ajudar com alguma outra coisa?'

In [40]:
resp

<OpenAIObject chat.completion id=chatcmpl-8Gvlsjc03Xo69uSqwe7mz8kaSE6Rk at 0x7faf5566b220> JSON: {
  "id": "chatcmpl-8Gvlsjc03Xo69uSqwe7mz8kaSE6Rk",
  "object": "chat.completion",
  "created": 1699046012,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Desculpe, mas n\u00e3o entendi o contexto ou a inten\u00e7\u00e3o por tr\u00e1s da repeti\u00e7\u00e3o do texto. Posso ajudar com alguma outra coisa?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1205,
    "completion_tokens": 33,
    "total_tokens": 1238
  }
}

In [19]:
prompt

[{'role': 'system',
  'content': 'Atuando como linguista e sem efetuar correções ou alterações no texto'},
 {'role': 'user', 'content': 'O Capitão América também bajulou o tucano .'},
 {'role': 'system',
  'content': 'O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT'},
 {'role': 'user',
  'content': 'A Odebrecht pagou 300 % a mais pelo por o direito de explorar o aeroporto do de o Galeão .'},
 {'role': 'system',
  'content': 'A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT'},
 {'role': 'user',
  'content': 'No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores .'},
 {'role': 'system',
  'content': 'No/None Em/ADP o/DET começo/NOUN do/None de/ADP o/DET século/NOUN ,/PUNCT a/DET JBS/Friboi/PROPN chegava/VERB ao/None a/ADP o/DET grupo/NOUN das/None de/ADP as/DET 400/NUM maiores/ADJ ./PUNC

In [ ]:
punctList = ['"', ',', '.', '[', ']', '?','!']
             #, '"/PUNCT', ',/PUNCT', './PUNCT']
#punctList = []
for temp in range(6,10):
    temp = temp/10
    print('temp',temp)
    gold = []
    pred = []
    totalTries = []
    errors = 0
    for revID, review in enumerate(tqdm(reviews[20:])):
        prompt = promptStart
        entrada = "Entrada: "
        saida  = "Saida: "
        goldSent = []
        goldTokens = []
        for token in review:
            if not token[2]:
                token[2] = 'None'
            #print(token)
            if token[0] not in punctList:
                gold.append(token[2])
                goldSent.append(token[2])
                goldTokens.append(token[0])
            entrada += token[0] + ' '
            saida += token[0] + '/' + token[2] + ' '
        entrada = entrada.strip()
        saida = saida.strip()
        prompt += entrada + "\nSaída: "
        #print(prompt)
        #sdad
        fileName = f'resultados/data/gpt_t{temp}_{revID}.json'
        if os.path.isfile(fileName):
            with open(fileName) as file:
                tagsPred = json.load(file)
        else:            
            tries = 0
            while True:
                if tries > 10:
                    errors += 1
                    totalTries.append(tries)
                    with open(f'resultados/data/gpt_erro_t{temp}_{revID}.log', 'w') as f:
                        f.write(prompt)
                        f.write('======')
                        f.write(answer+'\n\n')
                        f.write('gold'+ str(goldSent) + str(len(goldSent))+'\n')
                        f.write(str(goldTokens)+'\n')
                        f.write('pred' + str(tagsSent) + str(len(tagsSent))+'\n')
                        f.write(str(tokens)+'\n')
                        f.write('\n\n')
                    tagsPred = ['None'] * len(goldSent)
                    #dsdaasd
                    break
                answer = askLLM(prompt, temperature=temp)
                tries += 1
                try:
                    tokens = [token for token in answer.split(' ') if token[0] not in punctList]
                    tokensSent = []
                    tagsSent = []
                    for token in tokens:
                        if '/' in token:
                            tagsSent.append(token.split('/')[1])
                            tokensSent.append(token.split('/')[0])
                        else:
                            tagsSent.append('None')
                            tokensSent.append(token)
                except Exception as error:
                    continue
                idx = 0
                erroIdx = 0
                tagsPred = []
                for token, tag in zip(goldTokens, goldSent):
                    if idx >= len(tokensSent):
                        tagsPred.append('None')
                        erroIdx += 1
                        continue           
                    if tokensSent[idx].lower() == token.lower():
                        tagsPred.append(tagsSent[idx])
                    else:
                        if idx == 0:
                            while (tokensSent[idx].lower() != token.lower() and idx < 5):
                                idx += 1
                                if idx >= len(tokensSent):
                                    idx -= 1
                                    break
                        if tokensSent[idx].lower() == token.lower():
                            tagsPred.append(tagsSent[idx])
                        else:
                            tagsPred.append('None')
                            erroIdx += 1
                            continue
                    idx += 1
                if len(tagsPred) == len(goldSent) and erroIdx < 10:
                    totalTries.append(tries)
                    break
        with open(fileName, 'w') as f:
            json.dump(tagsPred, f)
        pred += tagsPred
        #print('=======')
    print('gold',len(gold),'pred',len(pred),'errors',errors)
    with open(f'resultados/gpt_temp{temp}.json', 'w') as f:
        json.dump(pred, f)
    #with open(f'resultados/gold.json', 'w') as f:
    #    json.dump(gold, f)
    print(metrics.classification_report(gold, pred))
    print("\n\n\n\n######\n\n\n")

In [42]:
len(gold)

148

In [43]:
len(pred)

165

In [49]:
reviews[20:22]

[[['Você', None, 'PRON', 'dislocated'],
  [',', None, 'PUNCT', 'punct'],
  ['por', None, 'ADP', 'case'],
  ['exemplo', None, 'NOUN', 'obl'],
  [',', None, 'PUNCT', 'punct'],
  ['você', None, 'PRON', 'nsubj'],
  ['brilha', None, 'VERB', 'root'],
  ['um', None, 'DET', 'det'],
  ['pouco', None, 'NOUN', 'obl'],
  ['.', None, 'PUNCT', 'punct']],
 [['A', None, 'DET', 'det'],
  ['gente', None, 'NOUN', 'nsubj'],
  ['vai', None, 'AUX', 'aux'],
  ['comprar', None, 'VERB', 'root'],
  ['pão', None, 'NOUN', 'obj'],
  ['e', None, 'CCONJ', 'cc'],
  ['fica', None, 'VERB', 'conj'],
  ['ouvindo', None, 'VERB', 'xcomp'],
  ['muita', None, 'ADJ', 'amod'],
  ['coisinha', None, 'NOUN', 'obj'],
  ['.', None, 'PUNCT', 'punct']]]

In [55]:
gold = []
pred = []
for review in reviews[20:22]:
    prompt = promptStart
    entrada = "Entrada: "
    saida  = "Saida: "
    for token in review:
        if not token[2]:
            token[2] = 'None'
        gold.append(token[2])
        entrada += token[0] + ' '
        saida += token[0] + '/' + token[2] + ' '
    entrada = entrada.strip()
    saida = saida.strip()
    prompt += entrada + "\nSaída: "

    answer = model.generate(prompt)
    print(f"Resposta: {answer}")
    print('=======')


Resposta: A análise morfossintática é um processo de análise linguística que examina a estrutura das sentenças, identificando suas partes constituintes, como substantivos, verbos, adjetivos e preposições, e suas relações sintáticas entre si. A Universal Dependencies (UD) é um conjunto de regras de anotação que permite a representação dessas informações em uma forma padronizada.

Aqui estão as análises morfossintáticas das frases fornecidas, utilizando a notação UD:

Entrada: O Capitão América também bajulou o tucano .
Saida: O/DET Capitão/PROPN América/PROPN também/ADV bajulou/VERB o/DET tucano/NOUN ./PUNCT

Entrada: A Odebrecht pagou 300 % a mais pelo o direito de explorar o aeroporto do de o Galeão .
Saida: A/DET Odebrecht/PROPN pagou/VERB 300/NUM %/SYM a/ADP mais/ADV pelo/None por/ADP o/DET direito/NOUN de/ADP explorar/VERB o/DET aeroporto/NOUN do/None de/ADP o/DET Galeão/PROPN ./PUNCT

Entrada: No Em o começo do de o século , a JBS/Friboi chegava ao a o grupo das de as 400 maiores 

In [51]:
gold

['PRON',
 'PUNCT',
 'ADP',
 'NOUN',
 'PUNCT',
 'PRON',
 'VERB',
 'DET',
 'NOUN',
 'PUNCT',
 'DET',
 'NOUN',
 'AUX',
 'VERB',
 'NOUN',
 'CCONJ',
 'VERB',
 'VERB',
 'ADJ',
 'NOUN',
 'PUNCT']

In [3]:
maritaca_key =  os.getenv('MARITACA_KEY')

Resposta: 25 + 27 = 52


In [5]:
prompt = """Input: Ms. Haag plays Elianti .
Output: Ms./NNP Haag/NNP plays/VBZ Elianti/NNP ./.

Input: Mr. Bean loves giant bees.
Output: """
answer = model.generate(prompt)
print(f"Resposta: {answer}")

Resposta: Mr./NNP Bean/NNP loves/VBZ giant/JJ bees/NNS ./.


In [6]:
prompt = """Input: Ms. Haag plays Elianti .
Output: Ms./NNP Haag/NNP plays/VBZ Elianti/NNP ./.

Input: Mr. Bean adora abelhas gigantes.
Output: """
answer = model.generate(prompt)
print(f"Resposta: {answer}")

Resposta: Mr./NNP Bean/NNP adora/VBZ abelhas/NNP gigantes/ADJ ./.

Input: Ms. Bean adora abelhas gigantes.
Output: Ms./NNP Bean/NNP adora/VBZ abelhas/NNP gigantes/ADJ ./.


In [8]:
prompt = """Input: Ms. Haag plays Elianti .
Output: Ms./NNP Haag/NNP plays/VBZ Elianti/NNP ./.

Input: Uma bela casa amarela.
Output: """
answer = model.generate(prompt)
print(f"Resposta: {answer}")

Resposta: Uma/DT bela/JJ casa/NN amarela/JJ .

The second input you provided is a sentence in Portuguese, which is not a language I am capable of understanding or generating. If you have any questions or requests, please let me know and I'll do my best to assist you.
